### Notebook used to primarily generate training dataset, using GLIMS, Google Earth Engine and geemap. Further down below are also the scripts
### used to generate evaluation data for the model.

In [ ]:
###################################################################################
# Only used to preprocess the GLIMS dataset. Only needs to be executed once!      #
#                                                                                 #
#                                                                                 # 
###################################################################################


# Import needed libraries
import sys
import dask_geopandas as dg
import pandas as pd
sys.path.append('..')
from scripts.data_processing_utils import DaskUtils, GISProcessor

# initialize Dask for filtering of the dataset

GISProcessor = GISProcessor()

# Get target columns
target_columns = ['line_type', 'glac_id', 'src_date', 'glac_stat', 'geometry']

# Read the dataset
fp = "/home/robin/Nextcloud_sn/QGIS/raw/glims_download_19805/glims_polygons.shp"

ddf = dg.read_file(fp, npartitions=6)
ddf = ddf[target_columns]

# Filter the dataset
df_bound_gone = ddf[(ddf["glac_stat"] == "gone") & (ddf['line_type'] == 'glac_bound')].compute() 
df_bound_exist = ddf[(ddf["glac_stat"] == "exists") & (ddf['line_type'] == 'glac_bound')].compute()


# Drop the line_type column
df_bound_gone.drop(columns=['line_type'], inplace=True)
df_bound_exist.drop(columns=['line_type'], inplace=True)

# Convert the src_date column to datetime
df_bound_gone['src_date'] = pd.to_datetime(df_bound_gone['src_date'])
df_bound_exist['src_date'] = pd.to_datetime(df_bound_exist['src_date'])

# Dissolve the dataframes
df_bound_exist_dissolved = GISProcessor.dissolve_glaciers(df_bound_exist)
df_bound_gone_dissolved = GISProcessor.dissolve_glaciers(df_bound_gone)

# Drop the glac_stat column
df_bound_exist_dissolved.drop(columns=['glac_stat'], inplace=True)
df_bound_gone_dissolved.drop(columns=['glac_stat'], inplace=True)

# Make a residual dataset
df_residual = GISProcessor.make_residuals(df_bound_exist_dissolved, min_change=5, max_change=40)

# Remove MultiPolygon geometries
df_residual['geometry'] = df_residual['geometry'].apply(GISProcessor.keep_largest_multipolygon)


# Save the filtered dataset
fp_out = "/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/test_refactor_code/03.07.2024"

#df_exists_bound.to_file(fp_out + "/exists_bound.shp")
df_bound_gone_dissolved.to_file(fp_out + "/gone_bound.shp")
df_residual.to_file(fp_out + "/residuals.shp")

In [ ]:
import geopandas as gpd
import pandas as pd
import sys
sys.path.append('..')
from scripts.data_processing_utils import DatasetMaker
import os
import geemap
import matplotlib.pyplot as plt
import numpy as np
import ee
import logging
import time


# Set paths and dataset version
version = "V1.6"

# Create the directories
base_data_path = f'/home/robin/Nextcloud_sn/Masterarbeit/DataSet/{version}'
sub_paths = ['DEMs', 'Inner-Outer Mask', 'Intersection Mask', 'Intersection Mask Small']

for sub_path in sub_paths:
    full_path = os.path.join(base_data_path, sub_path)
    os.makedirs(full_path, exist_ok=True)

# Set the paths
file_path_dem = f'/home/robin/Nextcloud_sn/Masterarbeit/DataSet/{version}/DEMs'
path_inner_outer = f'/home/robin/Nextcloud_sn/Masterarbeit/DataSet/{version}/Inner-Outer Mask'
path_intersection = f'/home/robin/Nextcloud_sn/Masterarbeit/DataSet/{version}/Intersection Mask'
path_intersection_small = f'/home/robin/Nextcloud_sn/Masterarbeit/DataSet/{version}/Intersection Mask Small'

# Set the logging level
logging.basicConfig(filename=f'/home/robin/Nextcloud_sn/Masterarbeit/DataSet/{version}/logging.txt',level=logging.ERROR)

# Initialize the Earth Engine
geemap.ee_initialize()

# Initialize the DatasetMaker
ds_maker = DatasetMaker()

# Read the saved datasets
df_residuals = gpd.read_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/test_refactor_code/03.07.2024/residuals.shp")
df_gone = gpd.read_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/test_refactor_code/03.07.2024/gone_bound.shp")

# Concatenate the datasets

df = pd.concat([df_residuals[['glac_id', 'geometry']], df_gone[['glac_id', 'geometry']]])
df.reset_index(drop = True, inplace = True)

df = ds_maker.filter_glaciers_by_country(df, ['Armenia', 'Azerbaijan'])
df.reset_index(drop = True, inplace = True)


# Mapping of masked pixels
value_mapping = {
    'Void (no data)': 0,
    'Edited (except filled pixel)': 1,
    'Not edited / not filled': 2,
    'ASTER': 3,
    'SRTM90': 4,
    'SRTM30': 5,
    'GMTED2010': 6,
    'SRTM30plus': 7,
    'TerraSAR-X Radargrammetric DEM': 8,
    'AW3D30': 9,
    'Norway DEM': 100,
    'DSM05 Spain': 101
}

df_flm = pd.DataFrame(columns=['glac_id', 'Void (no data)', 'Edited (except filled pixel)', 'Not edited / not filled', 'ASTER', 'SRTM90', 'SRTM30', 'GMTED2010',
                                  'SRTM30plus', 'TerraSAR-X Radargrammetric DEM', 'AW3D30', 'Norway DEM', 'DSM05 Spain', 'overall_sum'])


In [ ]:
# Supresses geemap output. However, it is hard to know, how far the download has progressed!
# To supress, just switch this function with the ee_export_image function in the cell below or vice versa.
# TQDM would be a better solution, however, it oftentimes halted the execution of the script.
import contextlib

def ee_export_image(*args, **kwargs):
    with open(os.path.join(base_data_path, 'logging_gee.txt'), 'a') as f:
        with contextlib.redirect_stdout(f):
            geemap.ee_export_image(*args, **kwargs)


In [ ]:
unique_ids = df['glac_id'].unique()
failed_ids = []
large_ids = []
small_ids = []
scale = 30
large_dim = 256
small_dim = 64



for glac_id in unique_ids:

    
    # Get the glacier dataframe
    
    df_glacid = df[df['glac_id'] == glac_id]
    
    # Same Filename for all tiff
    file_name = str(glac_id) + '.tif'
    # same filename for all masks
    file_name_mask = str(glac_id) + '.npy'

    # Values for the while loop, checking correct inner-outer mask generation
    tries_inner_outer = 0
    max_tries_inner_outer = 1000
    truth_value = True

    try:
        if ds_maker.glacier_area_check(scale, large_dim, df_glacid): # Area of glacier bounds is larger than area of mask
            
            # Get random bbox intersecting the glacier
            gdf_bbox_rand, gdf_intersection = ds_maker.get_random_bbox_intersect(df_glacid, buffer = 5000, min_coverage = 0.05, 
                                                                                 max_coverage = 0.9,img_res = large_dim, max_tries= 1000)

            if gdf_bbox_rand is None or gdf_intersection is None:
                failed_ids.append(glac_id)
                continue

            # Get random bbox again but in small scale (64 x 64)
            gdf_bbox_rand_small, gdf_intersection_small = ds_maker.get_random_bbox_intersect(gdf_intersection, spatial_resolution = scale, 
                                                                                             img_res = small_dim, buffer = 0, min_coverage = 0.05,
                                                                                             max_tries= 1000)

            if gdf_bbox_rand_small is None or gdf_intersection_small is None:
                failed_ids.append(glac_id)
                continue
                

            # Download and Save DEM
            path_abs = os.path.join(file_path_dem, file_name)        
            sq_geom = geemap.geopandas_to_ee(gdf_bbox_rand)
            filtered_band = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('DEM').filterBounds(sq_geom).mean()
            ee_export_image(filtered_band, filename=path_abs, scale=scale, region=sq_geom.geometry().bounds(), file_per_band=False, crs = 'EPSG:3857')

            # Get FLM table:
            flm_table = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('FLM').filterBounds(sq_geom).mean()
            flm_array = geemap.ee_to_numpy(flm_table, region=sq_geom.geometry().bounds(), scale=scale)

            # Set glac_id for which values are stored
            df_flm.loc[len(df_flm.index), 'glac_id'] = glac_id

            # Get unique values and their counts
            unique_labels, count = np.unique(flm_array, return_counts=True)

            # Store values in the dataframe
            for key in value_mapping.keys():
                df_flm.loc[df_flm.index[df_flm['glac_id'] == glac_id][0], key] = np.sum(count[unique_labels == value_mapping[key]])
                df_flm.loc[df_flm.index[df_flm['glac_id'] == glac_id][0], 'overall_sum'] = len(flm_array.flatten())

            # Generate masks
            inner_outer_mask, intersection_mask, intersection_mask_small = ds_maker.bbox_2_mask_rasterized(gdf_bbox_rand, gdf_bbox_rand_small, gdf_intersection, gdf_intersection_small,
                                                                                                           properties = {
                                                                                                            'outer_dim' : (large_dim, large_dim),
                                                                                                            'inner_dim' : (small_dim, small_dim),
                                                                                                        })
            # inner_outer_mask, intersection_mask, intersection_mask_small = Image.fromarray(inner_outer_mask), Image.fromarray(intersection_mask), Image.fromarray(intersection_mask_small)

            # Save masks
            np.save(os.path.join(path_inner_outer, file_name_mask), inner_outer_mask)
            np.save(os.path.join(path_intersection, file_name_mask), intersection_mask)
            np.save(os.path.join(path_intersection_small, file_name_mask), intersection_mask_small)

            large_ids.append(glac_id)
        

        elif not ds_maker.glacier_area_check(scale, large_dim, df_glacid): # Area of glacier bounds is smaller than area of mask

            gdf_bbox_rand, gdf_intersection = ds_maker.get_randomize_center_bbox(df_glacid, spatial_resolution=scale, img_res=large_dim)
            
            if ds_maker.glacier_area_check(scale, small_dim, gdf_intersection): # Area of glacier bounds is larger than area of mask
                gdf_bbox_rand_small, gdf_intersection_small = ds_maker.get_random_bbox_intersect(gdf_intersection, spatial_resolution = scale,
                                                                                                 img_res = small_dim, buffer = 0, min_coverage = 0.05,
                                                                                                 max_tries = 1000)

                # Chech if function workes
                if gdf_bbox_rand_small is None or gdf_intersection_small is None:
                    failed_ids.append(glac_id)
                    continue
                else:
                    # Generate masks
                    inner_outer_mask, intersection_mask, intersection_mask_small = ds_maker.bbox_2_mask_rasterized(gdf_bbox_rand, gdf_bbox_rand_small, gdf_intersection, gdf_intersection_small,
                                                                                                                   properties = {
                                                                                                            'outer_dim' : (large_dim, large_dim),
                                                                                                            'inner_dim' : (small_dim, small_dim),
                                                                                                        })

            else: # Area of glacier bounds is smaller than area of mask

                while tries_inner_outer < max_tries_inner_outer:
                    '''
                    Check if the sum of the inner and outer mask is equal to a specific dimension like 64*64. If not, try again.
                    If a correct result is archived, the loop will break. If not, the loop will break after 1000 tries. If it breaks after a 1000 tries, the glacier id will be added to the failed_ids list
                    and the entry will be skipped
                    '''

                    tries_inner_outer += 1

                    gdf_bbox_rand_small, gdf_intersection_small = ds_maker.get_randomize_center_bbox(gdf_intersection, spatial_resolution=scale, img_res=small_dim)

                    # Generate masks. Intersection mask is not used. We need another function here
                    inner_outer_mask, intersection_mask, intersection_mask_small = ds_maker.bbox_2_mask_rasterized(gdf_bbox_rand, gdf_bbox_rand_small, gdf_intersection, gdf_intersection_small,
                                                                                                                   properties = {
                                                                                                            'outer_dim' : (large_dim, large_dim),
                                                                                                            'inner_dim' : (small_dim, small_dim),
                                                                                                        })                    
                    if ds_maker.check_sum_inner_outer(inner_outer_mask, dim = (small_dim, small_dim)):
                        truth_value = False
                        break

                if truth_value:
                    failed_ids.append(glac_id)
                    continue

            path_abs = os.path.join(file_path_dem, file_name)        
            sq_geom = geemap.geopandas_to_ee(gdf_bbox_rand)
            filtered_band = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('DEM').filterBounds(sq_geom).mean()
            ee_export_image(filtered_band, filename=path_abs, scale=scale, region=sq_geom.geometry().bounds(), file_per_band=False, crs = 'EPSG:3857')

            # Get FLM table:
            flm_table = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('FLM').filterBounds(sq_geom).mean()
            flm_array = geemap.ee_to_numpy(flm_table, region=sq_geom.geometry().bounds(), scale=scale)

            # Set glac_id for which values are stored
            df_flm.loc[len(df_flm.index), 'glac_id'] = glac_id

            # Get unique values and their counts
            unique_labels, count = np.unique(flm_array, return_counts=True)

            # Store values in the dataframe
            for key in value_mapping.keys():
                df_flm.loc[df_flm.index[df_flm['glac_id'] == glac_id][0], key] = np.sum(count[unique_labels == value_mapping[key]])
                df_flm.loc[df_flm.index[df_flm['glac_id'] == glac_id][0], 'overall_sum'] = len(flm_array.flatten())

            # Transform masks in appropriate format        
            #inner_outer_mask, intersection_mask, intersection_mask_small = Image.fromarray(inner_outer_mask), Image.fromarray(intersection_mask), Image.fromarray(intersection_mask_small)

            # Save masks
            np.save(os.path.join(path_inner_outer, file_name_mask), inner_outer_mask)
            np.save(os.path.join(path_intersection, file_name_mask), intersection_mask)
            np.save(os.path.join(path_intersection_small, file_name_mask), intersection_mask_small)

            small_ids.append(glac_id)  


        else:
            print('Found glacier with equal bounds in id: ', glac_id)
    
    except Exception as e:
        logging.error(f"Error: {e} in glacier {glac_id}")
        print(f"Error: {e} in glacier {glac_id}")
        time.sleep(5)
        continue


    # Check if all files are saved correctly
    folder_paths = [file_path_dem, path_inner_outer, path_intersection, path_intersection_small]
    file_counts = []

    for folder in folder_paths:
        file_counts.append(len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))]))

    if len(set(file_counts)) != 1:  # Check if all counts are the same
        raise ValueError(f"Error: Mismatched file counts in folders after processing glacier {glac_id}: {file_counts}")
    

    with open(os.path.join(base_data_path, 'failed_ids.txt'), 'w') as f:
        for item in failed_ids:
            f.write("%s\n" % item)

    with open(os.path.join(base_data_path, 'large_ids.txt'), 'w') as f:
        for item in large_ids:
            f.write("%s\n" % item)

    with open(os.path.join(base_data_path, 'small_ids.txt'), 'w') as f:
        for item in small_ids:
            f.write("%s\n" % item)



df_flm.to_csv(os.path.join(base_data_path, 'flm_values.csv'), mode='w', index=False)

print('All files saved correctly\n Finished processing')


## Down below are cells used to evaluate model performanc

In [ ]:
######################################### Dataset for Evaluation Purposes #########################################

###################################################### Create gla_thi_da shapefile ######################################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

# Read the dataset
fp = "/home/robin/Nextcloud_sn/QGIS/raw/GlaThiDa/glathida-3.1.0/glathida-3.1.0/data/TTT.csv"

gla_thi_da = pd.read_csv(fp, sep=",")
gla_thi_da['SURVEY_DATE'] = gla_thi_da['SURVEY_DATE'].astype(str)
gla_thi_da['YEAR'] = gla_thi_da['SURVEY_DATE'].str[:4]
#gla_thi_da['YEAR'] = pd.to_datetime(gla_thi_da['YEAR'], format='%Y')

gla_thi_da.rename(columns={'POINT_LAT': 'lat', 'POINT_LON': 'lon', 'GLACIER_NAME': 'glac_name', 'YEAR': 'year'}, inplace=True)
gla_thi_da['glac_name'] = gla_thi_da['glac_name'].str.lower()
gla_thi_da['glac_name'] = gla_thi_da['glac_name'].str.replace(' ', '_')

gla_thi_da = gpd.GeoDataFrame(gla_thi_da, geometry=gpd.points_from_xy(gla_thi_da.lon, gla_thi_da.lat), crs='EPSG:4326')
gla_thi_da.rename_geometry('point_geom', inplace=True)

cols_to_drop = ['GlaThiDa_ID', 'POLITICAL_UNIT', 'SURVEY_DATE', 'PROFILE_ID', 'POINT_ID', 'lat', 'lon']
gla_thi_da.drop(columns= cols_to_drop, inplace=True)
gla_thi_da.reset_index(drop=True, inplace=True)

gla_thi_da.to_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/TTT.shp")

In [ ]:
###################################################### Create glims_polygons shapefile ######################################################
import dask_geopandas as dg
import sys
sys.path.append('..')
from scripts.data_processing_utils import GISProcessor
gis_processor = GISProcessor()
import pandas as pd

# Read the dataset
fp = "/home/robin/Nextcloud_sn/QGIS/raw/glims_download_19805/glims_polygons.shp"
# Get target columns
target_columns = ['line_type', 'glac_id', 'src_date', 'glac_stat', 'geometry', 'glac_name']

ddf = dg.read_file(fp, npartitions=6)
ddf = ddf[target_columns]

ddf = ddf[(ddf['glac_stat'] == 'exists') & (ddf['line_type'] == 'glac_bound')].compute()

ddf.drop(columns=['line_type'], inplace=True)
ddf['src_date'] = pd.to_datetime(ddf['src_date'])
ddf = gis_processor.dissolve_glaciers(ddf)
ddf.drop(columns=['glac_stat'], inplace=True)

ddf.rename_geometry('outline_glaciers', inplace=True)
ddf['glac_name'] = ddf['glac_name'].str.lower()
ddf['glac_name'] = ddf['glac_name'].str.replace(' ', '_')
ddf['year'] = ddf['year'].astype(str)

ddf.rename_geometry('polygon_geom', inplace=True)
ddf.reset_index(inplace=True)


ddf.to_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/glims_polygons.shp")

In [ ]:
###################################################### Create gla_rho shapefile ######################################################
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import dask_geopandas as dg

# Read the dataset
gla_rho = gpd.read_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/gla_rho.shp")
glims_rho = gpd.read_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/glims_rho.shp")

# Filter the data
gla_rho_2008 = gla_rho[gla_rho['year'] == '2008']
glims_rho_2008 = glims_rho[glims_rho['year'] == '2008']

# Reproject the data
gla_rho_2008 = gla_rho_2008.to_crs('EPSG:3857')
glims_rho_2008 = glims_rho_2008.to_crs('EPSG:3857')

# Spatial join
s_join = gpd.sjoin(gla_rho_2008, glims_rho_2008[['geometry']], how='inner', op='within')
s_join.head()

# Merge the data
merged = s_join.merge(glims_rho_2008[['geometry', 'glac_name']], how='left', on='glac_name')
merged.rename(columns = {'geometry_x': 'gla_rho_geom', 'geometry_y': 'glims_rho_geom'}, inplace=True)
merged.drop(columns=['index_right'], inplace=True)

# Create a GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged, crs='EPSG:3857', geometry=merged['gla_rho_geom'])
merged_gdf.drop(columns=['gla_rho_geom'], inplace=True)
merged_gdf.rename(columns={'geometry': 'gla_rho_geom'}, inplace=True)

# Set the geometry
merged_gdf.set_geometry('gla_rho_geom', inplace=True)

# Set glims to wkt. GeoDataFrame does not support more than one geometry column!
merged_gdf['glims_wkt'] = merged_gdf['glims_rho_geom'].to_wkt()
merged_gdf.drop(columns=['glims_rho_geom', 'glims_wkt'], inplace=True)
merged_gdf.reset_index(drop=True, inplace=True)

# Save the data
merged_gdf.to_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/eval_rhone.gpkg", driver='GPKG')

In [ ]:
###################################################### load rhone glacier ######################################################
import geopandas as gpd
import matplotlib.pyplot as plt
import geemap
import ee
import sys
sys.path.append('..')
from scripts.data_processing_utils import DatasetMaker

ds_maker = DatasetMaker()

# Read the dataset

df_rhone = gpd.read_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/eval_rhone.gpkg")


In [ ]:
####################################### code to get trift 2003 Glacier ########################################

import dask_geopandas as dg
import geopandas as gpd
import pandas as pd

dg_glims = gpd.read_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/glims_polygons.shp")
df_gla = gpd.read_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/TTT.shp")

dg_glims = dg_glims.to_crs('EPSG:3857')

glacier_to_check = ['gornergletscher',  'unteraargletscher', 'rhone', 'oberaletsch_gletscher',
                    'zmuttgletscher', 'findelengletscher', 'triftgletscher', 'turtmanngletscher',  'allalingletscher',
                    'gauligletscher']
mask = dg_glims['glac_name'].isin(glacier_to_check)
filtered_glims = dg_glims[mask]

mask = df_gla['glac_name'].isin(glacier_to_check)

filtered_glims['fits'] = filtered_glims['geometry'].apply(lambda x: ds_maker.glacier_fits_bbox_forDataframe(x, 30, 256))

glims_fits = filtered_glims[filtered_glims['fits'] == True]

#Triftgletscher 2003 seems to be the best fit
trift_2003 = glims_fits.loc[glims_fits['glac_name'] == 'triftgletscher']
trift_2003 = trift_2003.loc[(trift_2003['glac_id'] == 'G007685E46051N') & (trift_2003['year'] == '2003')]


trift_2003.to_file("/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/trift_2003.gpkg", driver='GPKG')
